# GAN: Generative Adversarial networks 

GANs are able to learn how to reproduce synthetic data that looks real. For instance,
computers can learn how to paint and create realistic images.

The key intuition of GAN can be easily considered as analogous to art forgery, which is the process
of creating works of art that are falsely credited to other, usually more
famous, artists. GANs train two neural nets simultaneously:
- The generator G(Z) makes the forgery, and 
- The discriminator D(Y) can judge how realistic the reproductions based on its observations of authentic pieces of arts and copies are.

D(Y) takes an
input, Y, (for instance, an image) and expresses a vote to judge how real the input is--in general, a
value close to zero denotes real and a value close to one denotes forgery. G(Z) takes an input from a
random noise, Z, and trains itself to fool D into thinking that whatever G(Z) produces is real. So, the
goal of training the discriminator D(Y) is to maximize D(Y) for every image from the true data
distribution, and to minimize D(Y) for every image not from the true data distribution. So, G and D
play an opposite game; hence the name adversarial training.

The generative model learns how to forge more successfully, and the discriminative
model learns how to recognize forgery more successfully.

The discriminator network (usually a
standard convolutional neural network) tries to classify whether an input image is real or generated.
The important new idea is to backpropagate through both the discriminator and the generator to adjust
the generator's parameters in such a way that the generator can learn how to fool the the discriminator
for an increasing number of situations. At the end, the generator will learn how to produce forged
images that are indistinguishable from real ones.

**Paper:** Unsupervised Representation Learning with
Deep Convolutional Generative Adversarial Networks, by A. Radford, L. Metz, and S. Chintala, 2015. -> https://arxiv.org/abs/1511.06434

### Deep Convolutional GAN
<img src="DCGAN.jpg">

### DCGAN applied to MNIST to forge the hand write numbers
https://github.com/jacobgil/keras-dcgan

https://github.com/bstriner/keras-adversarial

To use the code you need
- 1: Create a nre file named as dcgan.py 
- 2: Copy the code lines to this new file 



**Usage Training:**

        python dcgan.py --mode train --batch_size <batch_size>

        python dcgan.py --mode train --path ~/images --batch_size 128

**Image generation:**

        python dcgan.py --mode generate --batch_size <batch_size>

        python dcgan.py --mode generate --batch_size <batch_size> --nice : top 5% images according to discriminator

        python dcgan.py --mode generate --batch_size 128
      
**Note:** Once I only have insterest to run only the training: 
         
         python dcgan.py --mode train
         
**Note that** Training GANs could be
very difficult because it is necessary to find the equilibrium between two players. If you are
interested in this topic, I'd advise you to have a look at a series of tricks collected by practitioners (htt
ps://github.com/soumith/ganhacks):

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist #dataset
import numpy as np
from PIL import Image
import argparse
import math

In [ ]:
def generator_model():
    model = Sequential()
     
    """The first dense layer takes a vector of 100 dimensions as input and it
    produces 1,024 dimensions with the activation function tanh as the output.
    We assume that the input is sampled from a uniform distribution in [-1, 1]."""
    model.add(Dense(input_dim=100, output_dim=1024))   
    model.add(Activation('tanh'))                         
    
    """ The next dense layer produces data of 128 x 7 x 7 in the output using batch
    normalization (for more information refer to Batch Normalization: Accelerating Deep Network Training
    by Reducing Internal Covariate Shift, by S. Ioffe and C. Szegedy, arXiv: 1502.03167, 2014), 
    a technique that can help stabilize learning by normalizing the input to each unit to zero mean
    and unit variance. Batch normalization has been empirically proven to accelerate the training in
    many situations, reduce the problems of poor initialization, and more generally produce more accurate results."""
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    
    """The Reshape() module produces data of 128 x 7 x 7 
    (128 channels, 7 width, and 7 height), dim_ordering to tf, and a UpSampling() module that produces a
    repetition of each one into a 2 x 2 square."""
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    
    """The convolutional layer producing 64 filters on 5 x 5 convolutional kernels with the activation tanh,
    followed by a new UpSampling() """
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    
    """A final convolution with one filter, and on 5 x 5 convolutional kernels with the activation tanh."""
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    
    return model

In [ ]:
def discriminator_model():
    model = Sequential()
    
    model.add(Conv2D(64, (5, 5),padding='same',input_shape=(28, 28, 1)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [ ]:
def generator_containing_discriminator(g, d):
    model = Sequential()
    
    model.add(g) 
    d.trainable = False
    model.add(d)
    
    return model

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

In [ ]:
def train(BATCH_SIZE):
    # The code takes a standard MNIST image with the shape (1, 28, 28)
    (X_train, y_train), (X_test, y_test) = mnist.load_data()                        # load data
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    # X_train = X_train.reshape((X_train.shape, 1) + X_train.shape[1:])
    """
    Applies a convolution with 64 filters of size 5 x 5 with tanh as the activation function. 
    This is followed by a max-pooling operation of size 2 x 2 and by a further convolution 
    max-pooling operation. The last two stages are dense, with the final one being the prediction
    for forgery, which consists of only one neuron with a sigmoid activation function.
    """
    d = discriminator_model()                                                      # discriminator model
    g = generator_model()                                                          # generator model
    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    
    """
    For a chosen number of epochs, the generator and discriminator are in turn
    trained by using binary_crossentropy as loss function.
    """
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    for epoch in range(100):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            # We assume that the input is sampled from a uniform distribution in [-1, 1]
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100)) 
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            if index % 20 == 0:
                image = combine_images(generated_images)                           # combine images
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save(
                    str(epoch)+"_"+str(index)+".png")
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            """
            At each epoch, the generator makes a number of
            predictions (for example, it creates forged MNIST images) and the discriminator tries to learn after
            mixing the prediction with real MNIST images.
            """
            d.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 10 == 9:
                g.save_weights('generator', True)
                d.save_weights('discriminator', True)


In [ ]:
def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
        d = discriminator_model()
        d.compile(loss='binary_crossentropy', optimizer="SGD")
        d.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
        generated_images = g.predict(noise, verbose=1)
        d_pret = d.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = g.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args

if __name__ == "__main__":
    args = get_args()
    if args.mode == "train":
        train(BATCH_SIZE=args.batch_size)
    elif args.mode == "generate":
generate(BATCH_SIZE=args.batch_size, nice=args.nice)